# Turkish Dictionary(Rules) Based Sentiment Analysis

# Step 1: import required libs

In [1]:
import nltk
import string
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from classifier.dbsa import DictionaryBasedSentimentAnalyzer
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score,cross_validate
from sklearn.metrics import make_scorer, accuracy_score, f1_score, precision_score, recall_score

# Step 2: import  Verius NLP tools

In [2]:
from veriusapigateway import VeriUsAPIGateway
vu = VeriUsAPIGateway("API_KEY")

# Step 3: load the dataset

In [3]:
df = pd.read_csv('./sample_beyazperde_dataset.csv')

# Step 4: drop the "NAN" values from dataset

In [4]:
df.dropna(inplace=True)

# Step 5:  drop "Neutral" labeled data if there exists

In [5]:
df = df[df.target !="Neutral"]

# Step 6:  drop punctuations and lower the sentences

In [6]:
def drop_punctuations(raw_text):
    clean_data = [] 
    for text in raw_text:
        tokens = nltk.word_tokenize(text)
        tokens = [w.lower() for w in tokens]

        table = str.maketrans('', '', string.punctuation)
        words = [w.translate(table) for w in tokens]
        clean_data.append(" ".join([w for w in words]))
    
    return clean_data

In [7]:
## to avoid SettingWithCopyWarning, copy the original df as dfa and return the df
dfa = df.copy()
dfa["punctuations_dropped"] = drop_punctuations(dfa.text)
df = dfa.copy()
df.head()

,target,text,punctuations_dropped
1,Positive,Çok kaliteli bir film.Yönetmen niccol az sayıd...,çok kaliteli bir filmyönetmen niccol az sayıda...
2,Positive,EN İYİ TURK FİLMİ,en i̇yi̇ turk fi̇lmi̇
3,Positive,Robert Downey Jr. müthiş ötesi oynamış. Cidden...,robert downey jr müthiş ötesi oynamış cidden ...
4,Positive,"Yorum yazanlara dikkat ettimde, abartılı bulmu...",yorum yazanlara dikkat ettimde abartılı bulmu...
7,Positive,Benim için bir sinema klasiği. İlk izlediğimde...,benim için bir sinema klasiği i̇lk izlediğimd...


# Step 7:  normalize each sentence in the dataset

In [8]:
def normalizer(stopwords_dropped_text):
    normalized = []
    for sentence in stopwords_dropped_text:
        normalized.append(vu.get_normal(sentence))
    
    return normalized

In [9]:
df["normalized"] = normalizer(df.punctuations_dropped)
df.head()

,target,text,punctuations_dropped,normalized
1,Positive,Çok kaliteli bir film.Yönetmen niccol az sayıd...,çok kaliteli bir filmyönetmen niccol az sayıda...,çok kaliteli bir filmyönetmen nicole az sayıda...
2,Positive,EN İYİ TURK FİLMİ,en i̇yi̇ turk fi̇lmi̇,en i̇yi̇ türk film
3,Positive,Robert Downey Jr. müthiş ötesi oynamış. Cidden...,robert downey jr müthiş ötesi oynamış cidden ...,robert downey jr müthiş ötesi oynamış ciden bi...
4,Positive,"Yorum yazanlara dikkat ettimde, abartılı bulmu...",yorum yazanlara dikkat ettimde abartılı bulmu...,yorum yazanlara dikkat etimde abartılı bulmuşl...
7,Positive,Benim için bir sinema klasiği. İlk izlediğimde...,benim için bir sinema klasiği i̇lk izlediğimd...,benim için bir sinema klasiği ilk izlediğimde ...


# Step 8:  stem each sentence tokens in the dataset

In [10]:
def stemmer(normalized_text):
    stemmed = []
    for sentence in normalized_text:
        stemmed.append(vu.get_stem(sentence))
    
    return stemmed

In [11]:
df["stemmed"] = stemmer(df.normalized)
df.head()

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (

,target,text,punctuations_dropped,normalized,stemmed
1,Positive,Çok kaliteli bir film.Yönetmen niccol az sayıd...,çok kaliteli bir filmyönetmen niccol az sayıda...,çok kaliteli bir filmyönetmen nicole az sayıda...,çok kalite bir filmyönet nicole az sayı ama ço...
2,Positive,EN İYİ TURK FİLMİ,en i̇yi̇ turk fi̇lmi̇,en i̇yi̇ türk film,en i tÃ¼rk film
3,Positive,Robert Downey Jr. müthiş ötesi oynamış. Cidden...,robert downey jr müthiş ötesi oynamış cidden ...,robert downey jr müthiş ötesi oynamış ciden bi...,robert downey jr müthiş öte oyna ciden bir süp...
4,Positive,"Yorum yazanlara dikkat ettimde, abartılı bulmu...",yorum yazanlara dikkat ettimde abartılı bulmu...,yorum yazanlara dikkat etimde abartılı bulmuşl...,yor yaza dikkat eti abart bul film sen çizgi r...
7,Positive,Benim için bir sinema klasiği. İlk izlediğimde...,benim için bir sinema klasiği i̇lk izlediğimd...,benim için bir sinema klasiği ilk izlediğimde ...,ben iç bir sine klasik ilk izle nefes kes adet...


# Step 9:  drop unnecessary columns in the dataset

In [12]:
df = df.drop(['text', 'punctuations_dropped', 'normalized'], axis=1)
df.head()

,target,stemmed
1,Positive,çok kalite bir filmyönet nicole az sayı ama ço...
2,Positive,en i tÃ¼rk film
3,Positive,robert downey jr müthiş öte oyna ciden bir süp...
4,Positive,yor yaza dikkat eti abart bul film sen çizgi r...
7,Positive,ben iç bir sine klasik ilk izle nefes kes adet...


# Step 10:  create an instance of "DictionaryBasedSentimentAnalyzer" classifier

In [13]:
model = DictionaryBasedSentimentAnalyzer()
model.fit()

# Step 11: take all dataset as test data

In [14]:
## since there the model is pretrained we take all data as test data
X_test = df.stemmed
## convert targets to numbers since cross_validate works with numbers
y_test = df.target.replace("Positive", 1).replace("Negative", 0)

# Step 12:  predict the test data

In [15]:
y_pred = model.predict(X_test)

# Step 13:  observe the accuracy score

In [16]:
model.score(y_test)

0.7618543046357615

# Step 14:  create a classification_report

In [17]:
target_names = df.target.unique()
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    Positive       0.27      0.24      0.25       637
    Negative       0.85      0.87      0.86      3138

   micro avg       0.76      0.76      0.76      3775
   macro avg       0.56      0.55      0.56      3775
weighted avg       0.75      0.76      0.76      3775



# Step 15:  observe the confusion matrix

In [18]:
confusion_matrix(y_test, y_pred)

array([[ 153,  484],
       [ 415, 2723]], dtype=int64)

# Step 16:  observe the prediction probabilities of the wrong predictions

In [19]:
pred_prob = model.predict_proba()
data = {'predict_proba': pred_prob,'pred': y_pred, 'real': y_test,'stemmed': X_test}
df_pred_prob = pd.DataFrame(data=data)
df_pred_prob = df_pred_prob[df_pred_prob.pred != df_pred_prob.real]
df_pred_prob.head()

,predict_proba,pred,real,stemmed
9,0.555556,0,1,film bit hala tam ne ol bitik ayık deği ama ho...
10,0.541096,0,1,sine tarih gel geç en iyi film tartış ama bu s...
22,0.611111,0,1,öyle çok gül ki ev ol gerek bir animasyon
23,0.600000,0,1,hey re already her
26,0.688334,1,0,201 yıl türkiy de sine salon yaklaş 72 milyon ...


# Step 17:  observe the roc auc curve

In [20]:
roc_auc_score(y_test, pred_prob)

0.6090023742987414

# Step 18:  apply ten-fold cross validation

In [21]:
scoring_list = ["f1_macro","precision_macro","recall_macro","accuracy"]
scores = cross_validate(model, X_test, y_test, cv=10, scoring=scoring_list)
for scr in scoring_list:
    print(scr+":"+"{:.2f}".format(scores["test_"+scr].mean()))

f1_macro:0.56
precision_macro:0.56
recall_macro:0.55
accuracy:0.76


# Step 19:  create pipeline for the model

In [22]:
model = Pipeline([
    ('model', model),
])

# Step 20:  pickle the model

In [24]:
joblib.dump(model,'Turkish_Dictionary(Rules)_Based_Sentiment_Analysis.pkl')

['Turkish_Dictionary(Rules)_Based_Sentiment_Analysis.pkl']